In [94]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import statsmodels as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from statsmodels.tsa.arima_model import ARMA, ARIMA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
with open('data_processed/data_feat.pickle', 'rb') as handle:
    data_feat = pickle.load(handle)
X_train = data_feat[0]
y_train = data_feat[1]
X_test = data_feat[2]
y_test = data_feat[3]

In [83]:
col_names = list(X_train.columns)
col_names

['step',
 'spool',
 'speed',
 'motorV',
 'extfrq',
 'temp',
 'd3',
 'inv_sqrt_w',
 'inv_sqrt_w2',
 'mean_speed_roll_10',
 'sum_speed_roll_10',
 'mean_extfrq_roll_10',
 'sum_extfrq_roll_10']

### Normalize Data

In [53]:
scaler_X = StandardScaler()
X_train_n = scaler_X.fit_transform(X_train)
X_test_n = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_n = scaler_y.fit_transform(np.array(y_train).reshape(-1,1))
y_test_n = scaler_y.transform(np.array(y_test).reshape(-1,1))

## Regression Models


In [54]:
### LASSO
lasso = LassoCV(cv=5, random_state=0, max_iter=10e4).fit(X_train_n, y_train_n)
train_rsq = lasso.score(X_train_n, y_train_n)
print('Train R-Squared: ', train_rsq)

y_hat = lasso.predict(X_test_n)
test_rsq = lasso.score(X_test_n, y_test_n)
print('Test R-Squared: ', test_rsq)

/Users/mschoder/.pyenv/versions/dev_374/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:1100: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train R-Squared:  0.5732725136911497
Test R-Squared:  0.5793971387553365


In [79]:
lasso.alpha_

0.00026706155288369127

In [65]:
y_train_n.ndim

2

## ARMAX 

In [95]:
model = ARIMA(y_train_n, exog=X_train_n, order=(2, 2, 2))
model_fit = model.fit(disp=True)

/Users/mschoder/.pyenv/versions/dev_374/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [82]:
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                51318
Model:                     ARMA(2, 2)   Log Likelihood               14016.511
Method:                       css-mle   S.D. of innovations              0.184
Date:                Mon, 02 Dec 2019   AIC                         -27995.023
Time:                        06:56:42   BIC                         -27826.953
Sample:                             0   HQIC                        -27942.439
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.524e-05      0.019      0.002      0.998      -0.037       0.037
x1            -0.2686      0.043     -6.202      0.000      -0.354      -0.184
x2            -0.0051      0.040     -0.127      0.8

In [84]:
# model_fit.predict(y_train, exog=X_train)

## Shift/Lag Fits

In [92]:
X_train = X_train[['step', 'spool', 'speed', 'extfrq', 'temp', 'inv_sqrt_w']]

speed_lag = 2
speed_sd = 2
extfrq_lag = 2
extfrq_sd = 2

X_train_p = 

,step,spool,speed,extfrq,temp,inv_sqrt_w
0,100,3.57,85.84,82.86,79.692,0.107933
1,101,3.61,85.28,83.06,79.727,0.108287
2,102,3.65,84.88,83.19,79.589,0.108542
3,103,3.68,84.32,83.17,79.727,0.108902
4,104,3.72,84.24,83.33,79.658,0.108953
...,...,...,...,...,...,...
51313,2577,99.79,88.80,80.55,80.002,0.106119
51314,2578,99.82,88.56,80.39,80.002,0.106263
51315,2579,99.86,88.80,80.27,80.002,0.106119
51316,2580,99.90,88.96,80.16,80.002,0.106024
